In [1]:
import pandas as pd

In [2]:
#patient_monthwise_revenue_test = pd.read_csv('data/patient_monthwise_revenue_test.csv')
patient_monthwise_revenue_train = pd.read_csv('data/patient_monthwise_revenue_train.csv')
physio_diagnosis_train = pd.read_csv('data/physio_diagnosis_train.csv')
#physio_appts_train = pd.read_csv('data/physio_appts_train.csv')
patient_train_classified = pd.read_csv('data/patient_train_classified.csv')

/home/pradeep/softs/anaconda3/envs/portea/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
patient_monthwise_revenue_train.info()

In [4]:
#physio_diagnosis_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48804 entries, 0 to 48803
Data columns (total 9 columns):
patientId     48804 non-null int64
diagnosis     47748 non-null object
Service       48804 non-null object
Appt_id       48803 non-null object
AGE           48804 non-null object
gender        48775 non-null object
LVD           48804 non-null object
Unnamed: 7    18 non-null object
Unnamed: 8    2 non-null object
dtypes: int64(1), object(8)
memory usage: 3.4+ MB


In [3]:
# Removing unnecessary data

# FVS - doesnt have proper information?
# diagnosis - duplicate data with 'physio_diagnosis_train'
# revenue - much valueable info in 'patient_train_classified', 
# may be if it is missing in train_classified for particular id, we need to take the value from here ?
# service_name - since we have service_id
# approx_age - since we have AGE in physio_diagnosis_train
# Unnamed: 20 - not required
# Unamed: 21 - not required
patient_monthwise_revenue_train = patient_monthwise_revenue_train.drop(columns=
                                                                       ['FVS', 'diagnosis', 
                                                                        'revenue', 'service_name', 'approx_age',
                                                                        'Unnamed: 20', 'Unnamed: 21'])

In [4]:
# gender - it is there in patient_monthwise_revenue_train
# LVD - it is there in patient_monthwise_revenue_train
# Service - we have service_id in patient_monthwise_revenue_train
# Unnamed: 7 - not required
# Unamed: 8 - not required
# Appt_id - since there is no mapping of training data we are loading for analysis
physio_diagnosis_train = physio_diagnosis_train.drop(columns=
                                                     ['gender', 'LVD', 'Unnamed: 7', 'Unnamed: 8', 'Service', 'Appt_id'])

In [7]:
# Nothing to remove from patient_train_classified

In [5]:
physio_diagnosis_train = physio_diagnosis_train.rename(columns={'patientId': 'patient_id'})
#physio_appts_train = physio_appts_train.rename(columns={'pat_id': 'patient_id'})
patient_train_classified = patient_train_classified.rename(columns={'PID': 'patient_id'})

In [6]:
md_revenue_diagnosis_train = pd.merge(patient_monthwise_revenue_train, physio_diagnosis_train, on='patient_id')
# Commenting out appts_train(appointments) data since the appts_test data is not available.
#md_revenue_diagnois_appts_train = pd.merge(md_revenue_diagnosis_train, physio_appts_train, on='patient_id')
final_raw_train = pd.merge(md_revenue_diagnosis_train, patient_train_classified, on='patient_id')

In [7]:
final_raw_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83307 entries, 0 to 83306
Data columns (total 19 columns):
patient_id          83307 non-null int64
visit_month_year    83307 non-null object
service_id          83307 non-null int64
visits_count        83307 non-null int64
city                83307 non-null object
ref_type            83307 non-null object
ref_name            83307 non-null object
ref_source          83307 non-null object
FVD                 83307 non-null object
FVM                 83307 non-null object
gender              83274 non-null object
LVD                 83307 non-null object
brand               83307 non-null object
visits_required     83307 non-null int64
avg_nps             83307 non-null object
diagnosis           81570 non-null object
AGE                 83307 non-null object
Bucket              83307 non-null object
Revenue             83307 non-null float64
dtypes: float64(1), int64(4), object(14)
memory usage: 12.7+ MB


In [ ]:
'''
final_raw_train = md_revenue_diagnois_appts_class_train.drop(columns=
    ['Unnamed: 20', 'Unnamed: 21','Unnamed: 7','Unnamed: 8',
     'serviceId', 'name','service_name','Service',
     'Appt_id', 'FVS', 'approx_age',
     'LVD_y', 'gender_y', 'revenue',
    'diagnosis_x'], axis=1)
'''

In [ ]:
#final_raw_train = final_raw_train.rename(columns={'gender_x': 'gender', 'LVD_x':'LVD', 'diagnosis_y': 'diagnosis'})

In [20]:
final_raw_train.sample(5)

,patient_id,visit_month_year,service_id,visits_count,city,ref_type,ref_name,ref_source,FVD,FVM,gender,LVD,brand,visits_required,avg_nps,diagnosis,AGE,Bucket,Revenue
5447,37738,11/14,2,7,Chennai,B2B,Apsalom S,Chemist + Doctors,03/11/2014,11/14,female,19/12/2017,Portea,15,9.25,post traumatic disorder,59,High,302250.0
81485,433969,10/17,1,1,Bangalore,B2C,SMS,Referral,23/10/2017,10/17,female,02/11/2017,Portea,10,10,ORIF and bone grafting post non union fracture...,45,Med,4800.0
21562,86324,07/15,2,8,Bangalore,B2C,Facebook Organic,Digital + ATL,03/07/2015,07/15,female,12/07/2015,Portea,7,7.67,"mechanical back pain ,SLR test :-ve, \ntendern...",30,Med,4000.0
67932,325213,03/17,2,3,Bangalore,B2C,None,Direct Organic,25/03/2017,03/17,male,08/04/2017,Portea,10,9,Stroke,60,Med,2750.0
13180,57465,03/15,2,9,NCR,B2C,Other,Direct Organic,11/03/2015,03/15,female,07/04/2015,Portea,10,\N,"osteo-arthritis,others - ortho",57,High-Med,6100.0


In [12]:
# Missing values
missing_values_count = final_raw_train.isnull().sum()

In [13]:
missing_values_count

patient_id             0
visit_month_year       0
service_id             0
visits_count           0
city                   0
ref_type               0
ref_name               0
ref_source             0
FVD                    0
FVM                    0
gender                33
LVD                    0
brand                  0
visits_required        0
avg_nps                0
diagnosis           1737
Appt_id               15
AGE                    0
Bucket                 0
Revenue                0
dtype: int64

In [8]:
# missing :1737  diagnosis, 33 Gender
# Applicable when merging with appts_trai data: Gender has 204 missing values and diagnosis has 40.85 K missing value
final_raw_train = final_raw_train.fillna(0)

In [ ]:
# Individual info about the dataframe with columns info
final_raw_train.info()

In [22]:
# Writing the raw data to csv file, the processed file
#final_raw_train.to_csv('data/full_train_pros.csv', index=False)
final_raw_train.to_csv('data/full_train_wo_appts_pros.csv', index=False)

In [9]:
#full_train_pros = pd.read_csv('data/full_train_pros.csv')
full_train_pros = pd.read_csv('data/full_train_wo_appts_pros.csv')

In [10]:
full_train_pros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83307 entries, 0 to 83306
Data columns (total 19 columns):
patient_id          83307 non-null int64
visit_month_year    83307 non-null object
service_id          83307 non-null int64
visits_count        83307 non-null int64
city                83307 non-null object
ref_type            83307 non-null object
ref_name            83307 non-null object
ref_source          83307 non-null object
FVD                 83307 non-null object
FVM                 83307 non-null object
gender              83307 non-null object
LVD                 83307 non-null object
brand               83307 non-null object
visits_required     83307 non-null int64
avg_nps             83307 non-null object
diagnosis           83307 non-null object
AGE                 83307 non-null object
Bucket              83307 non-null object
Revenue             83307 non-null float64
dtypes: float64(1), int64(4), object(14)
memory usage: 12.1+ MB


In [11]:
cols_to_process = ['visit_month_year','city','ref_type','ref_name',
                     'ref_source','FVD','FVM','gender','LVD','brand',
                     'avg_nps','diagnosis','AGE','Bucket']

In [12]:
# Now converting the object data into numerical data for analysis
# For 'visit_month_year'
'''
full_train_pros[['visit_month_year','city','ref_type','ref_name',
                     'ref_source','FVD','FVM','gender','LVD','brand',
                     'avg_nps','diagnosis','AGE','status','cost',
                     'discount','patient_ivr_confirmation','Bucket']] = full_train_pros[['visit_month_year','city','ref_type','ref_name',
                     'ref_source','FVD','FVM','gender','LVD','brand',
                     'avg_nps','diagnosis','AGE','status','cost',
                     'discount','patient_ivr_confirmation','Bucket']].astype(str)
'''
full_train_pros[cols_to_process] = full_train_pros[cols_to_process].astype(str)

In [13]:
# For encoding the strings
from sklearn.preprocessing import LabelEncoder


In [14]:
cols_with_classes = {}
for colm in cols_to_process:
    le = LabelEncoder()
    full_train_pros[colm] = le.fit_transform(full_train_pros[colm])
    cols_with_classes[colm] = le

In [38]:
#full_train_pros.to_csv('data/full_train_cleaned_encoded.csv', index=False)
#le.classes_
full_train_pros.to_csv('data/full_train_wo_appts_cleaned_encoded.csv', index=False)

In [26]:
#train_data = pd.read_csv('data/full_train_cleaned_encoded.csv')
train_data = pd.read_csv('data/full_train_wo_appts_cleaned_encoded.csv')

In [27]:
train_data = train_data.drop_duplicates()

In [28]:
# Getting the labels, 'Revenue' is not taken into consideration, may be that can be used for Regresssion
patient_ids = train_data['patient_id']
Y_train = train_data['Bucket']
Y_train_R = train_data['Revenue']

In [21]:
#X_train = train_data.drop(columns=['Bucket','Revenue', 'startDateTime']) # with appts_train
#X_train = train_data.drop(columns=['Bucket','Revenue'])
# Need to remove patient_id also 
X_train = train_data.drop(columns=['patient_id', 'Bucket','Revenue'])

In [22]:
# Converting into numpy array
X_train = X_train.as_matrix()
Y_train = Y_train.as_matrix()
Y_train_R = Y_train_R.as_matrix()

In [23]:
# Importing Regressor and Classifier from Support Vector Machine
from sklearn.svm import SVR, SVC

In [24]:
# Initializing classifier
clf = SVC()

In [25]:
# Running the Machine learning model for fit method.
clf.fit(X_train, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [31]:
# Initializing Regressor
reg = SVR()
# Calling the model to train on the data.
reg.fit(X_train, Y_train_R)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [29]:
# Saving the classifer into pickle format.
from six.moves import cPickle as pickle

try:
    # Saving the classifier to pickle
    with open('data/train_svc_clf_wo_pat_id.pickle', 'wb') as f:
        pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)
    
    # Saving Regressor to pickle
    '''
    with open('data/train_svr_reg.pickle', 'wb') as f:
        pickle.dump(reg, f, pickle.HIGHEST_PROTOCOL)
    '''    
    # Saving cols_with_classes to pickle which are needed for test
    '''with open('data/cols_with_classes.pickle', 'wb') as f:
        pickle.dump(cols_with_classes, f, pickle.HIGHEST_PROTOCOL)'''
except Exception as e:
    print('Got exception', e)

In [32]:
# Saving the classifer into pickle format.
from six.moves import cPickle as pickle

try:
    # Saving Regressor to pickle
    with open('data/train_svr_reg.pickle', 'wb') as f:
        pickle.dump(reg, f, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Got exception', e)